## Data exploration

In [ ]:
import os
import glob
import pandas as pd
import numpy as np
from PIL import Image, ImageChops

In [ ]:
print(str(len(glob.glob("./training/11390/2012_01_05_17_06_01_0/*"))) + " Images for sample 1")
print(str(len(glob.glob("./training/*/*"))) + " Different Samples")
# Avg number of images per sample
print(str(len(glob.glob("./training/*/*/*"))/len(glob.glob("./training/*/*"))) + " Average number images per sample")

In [ ]:
# Data for a single sample
sample = []
for i in glob.glob("./training/11390/2012_01_05_17_06_01_0/*"):
    print(i)
    sample.append(Image.open(i))

In [ ]:
for i in glob.glob("./training/11390/2012_01_05_17_06_01_0/*_continuum.jpg"):
    print(i)

In [ ]:
for i in glob.glob("./training/11390/2012_01_05_17_06_01_0/*_magnetogram.jpg"):
    print(i)

In [ ]:
for i in glob.glob("./training/11390/2012_01_05_17_06_01_0/*_211.jpg"):
    print(i)

In [ ]:
# _# represents AIA wavelength for band #
# Hour Times: 05, 12, 15, 16
# Not sure what continuum images represent
for i in glob.glob("./training/11390/2012_01_05_17_06_01_0/*_304.jpg"):
    print(i)

In [ ]:
sample[14]

In [ ]:
img = sample[17]

In [ ]:
Image.open("./training/11390/2012_01_05_17_06_01_0/2012-01-05T153601__magnetogram.jpg")

In [ ]:
print(img.format)
print(img.mode)

In [ ]:
img

In [ ]:
#8E-07 is peak flux for this sample
# Data Transformation 1: 
i = img.split()[0]
len(i.histogram())

## Baseline network
This network should directly predict peak flux

In [ ]:
import torch
from torch import nn

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv3d(10, 32, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(32 * 256 * 256 * 4, 128)
        self.fc2 = nn.Linear(128, 3)

    def forward(self, x):
        x = nn.functional.relu(self.conv1(x))
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Create the model
model = SimpleCNN()

## Temperature Model
This model predicts temperature and then converts at the end to flux

In [ ]:
class SimpleCNNWithWien(nn.Module):
    def __init__(self):
        super(SimpleCNNWithWien, self).__init__()
        # Adjusted convolutional layers for 3D data
        self.conv1 = nn.Conv3d(in_channels=4, out_channels=16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv3d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv3d(32, 64, 3, padding=1)

        # Adjusted pooling layers
        self.pool3d = nn.MaxPool3d(2, 2)
        
        # Linear layers
        # Update the input features of fc1 according to the output size of the last pooling layer
        # Example dimensions: 64 * 32 * 32 * 1 = 65536, if the final output size is [64, 32, 32, 1]
        self.fc1 = nn.Linear(64 * 32 * 32, 512)  # Adjust these dimensions
        self.fc2 = nn.Linear(512, 1)  # Outputting temperature
        self.dropout = nn.Dropout(0.25)

        # Wien's displacement constant (meters * Kelvin)
        self.wien_constant = 2.897771955e-3

    def forward(self, x):
        # Add sequence of convolutional and max pooling layers
        x = self.pool3d(F.relu(self.conv1(x)))
        x = self.pool3d(F.relu(self.conv2(x)))
        x = self.pool3d(F.relu(self.conv3(x)))

        # Flatten image input
        x = x.view(-1, 64 * 32 * 32)  # Update these dimensions accordingly
        x = self.dropout(x)

        # Add 1st hidden layer, with relu activation function
        x = F.relu(self.fc1(x))
        x = self.dropout(x)

        # Add 2nd hidden layer (outputs temperature)
        temperature = self.fc2(x)

        # Apply Wien's equation to calculate peak wavelength
        # Prevent division by zero in case temperature is zero
        temperature = torch.clamp(temperature, min=1e-6)
        peak_wavelength = self.wien_constant / temperature

        return peak_wavelength


## Data extraction
A single input will be a 4x256x256x10 matrix where dimensions represent: [timeinterval x height x width x wavelength/magnetogram]

In [ ]:
from tqdm import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from torch.utils.data import TensorDataset, DataLoader

# Get a list of all active region numbers
all_files = glob.glob("./training/*/*")

# Split all_files into 10 chunks
chunks = np.array_split(all_files, 40)
print(len(chunks[0]))

def generateChunk(idxs):
    wavelengths = ["94","131", "171","193","211","304","335","1700","continuum","magnetogram"]
    x = np.zeros((len(idxs), 4, 256, 256, 10), dtype=np.int64)
    y = np.zeros((len(idxs), 1), dtype=np.int64)  # Updated data type to int64
    df = pd.read_csv('training/meta_data.csv')
    peak_flux_dict = {idx: peak_flux for idx, peak_flux in zip(df['id'], df['peak_flux'])}

    for i, sample in enumerate(idxs):
        images = np.empty((4, 256, 256, 10), dtype=np.int64)
        for j, wave in enumerate(wavelengths):
            path = sample + "/*_{}.jpg".format(wave)
            pics = np.array([np.array(Image.open(i)) for i in glob.glob(path)])
            for _ in range(4 - len(pics)):
                if len(pics) == 0:
                    pics = np.zeros((1, 256, 256))
                else:
                    pics = np.concatenate((pics, np.zeros((1, 256, 256))), axis=0)
            pics = pics.reshape(4, 256, 256, 1)
            images[:, :, :, j] = pics[:, :, :, 0]
        images = images[:, :, :, :]
        x[i] = images

        idx = path.split("/")[2] + "_"+ path.split("/")[3]
        peak_flux = peak_flux_dict[idx]

        if peak_flux < 1e-5:
            y[i] = 0  # Common
        elif peak_flux < 1e-4:
            y[i] = 1  # Moderate
        else:
            y[i] = 2  # Extreme

    return torch.from_numpy(x.transpose(0,4,2,3,1)), torch.from_numpy(y)

#for i in range(40):
#    train_x, train_y = generateChunk(chunks[i])
#    dataset = TensorDataset(train_x, train_y)
#    torch.save(dataset, "datachunks/chunk_{}.pt".format(i))

## Create a DataLoader
#val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=True)

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

def train_model(model, criterion, optimizer, epochs):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)  # Move the model to the GPU if available

    for epoch in range(epochs):
        for i in range(39):
            train_x, train_y = generateChunk(chunks[i])
            dataset = TensorDataset(train_x, train_y)
            train_loader = DataLoader(dataset, batch_size=32, shuffle=True)
            model.train()  # Set the model to training mode
            for inputs, targets in train_loader:
                inputs, targets = inputs.to(device).float(), targets.to(device).long().reshape(-1)  # Move data to GPU
                optimizer.zero_grad()  # Reset the gradients
                outputs = model(inputs)  # Forward pass
                loss = criterion(outputs, targets)  # Compute loss
                loss.backward()  # Backward pass
                optimizer.step()  # Update weights

        train_x, train_y = generateChunk(chunks[39])
        dataset = TensorDataset(train_x, train_y)
        val_loader = DataLoader(dataset, batch_size=32, shuffle=True)
        model.eval()  # Set the model to evaluation mode
        with torch.no_grad():  # No need to track gradients in validation phase
            total_loss = 0
            total_correct = 0
            total_samples = 0
            for inputs, targets in val_loader:
                inputs, targets = inputs.to(device).float(), targets.to(device).long().reshape(-1)  # Move data to GPU
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                total_loss += loss.item() * len(targets)
                _, predicted = torch.max(outputs, 1)
                total_correct += torch.sum(predicted == targets)
                total_samples += len(targets)
        
        avg_loss = total_loss / total_samples
        accuracy = total_correct / total_samples
        print(f'Epoch {epoch+1}/{epochs}, Loss: {avg_loss}, Accuracy: {accuracy}')

mod = SimpleCNN()
train_model(mod, nn.CrossEntropyLoss(), torch.optim.Adam(mod.parameters()), 10)
mod = SimpleCNNWithWien()
train_model(mod, nn.CrossEntropyLoss(), torch.optim.Adam(mod.parameters()), 10)
